# Structured Output & Мультимодальные & Локальные модели

Практическое руководство по работе с современными возможностями LLM

## Содержание:
1. **Structured Output - Извлечение информации**
2. **Vision Models - Анализ изображений**
3. **Локальные модели - Теория**
4. **Локальные модели - Практика**


---
## Часть 1: Structured Output - Извлечение информации

### Задача
Создаем финансового советника, который извлекает транзакции из текста пользователя.

**Пример входа:** "Сегодня купил продукты на 1500 рублей"

**Пример выхода:**
```json
{
  "transactions": [{"date": "2024-10-30", "type": "expense", "amount": 1500.0, ...}],
  "answer": "Записал расход на продукты 1500₽"
}
```


In [1]:
# Установка зависимостей
%pip install -q openai>=1.54.0 pydantic>=2.0.0 python-dotenv>=1.0.0


zsh:1: 1.54.0 not found
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### 1.1. Старый подход: JSON через промпт (без schema)

**Идея:** просим модель вернуть JSON в промпте, надеемся на лучшее


In [3]:
from openai import OpenAI
import json
import os

# Настройка клиента
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY", "your-key"),
    base_url="https://openrouter.ai/api/v1"
)

# Промпт с просьбой вернуть JSON
system_prompt_old = """Ты финансовый советник. Извлеки транзакции из сообщения.

Верни ответ СТРОГО в JSON формате:
{
  "transactions": [{"amount": 1500.0, "category": "продукты"}],
  "answer": "текстовый ответ"
}
"""

user_message = "Купил продукты на 1500 рублей"

response = client.chat.completions.create(
    model="openai/gpt-4o-mini",  # или любая другая модель
    messages=[
        {"role": "system", "content": system_prompt_old},
        {"role": "user", "content": user_message}
    ],
    temperature=0.1  # низкая температура для предсказуемости
)

raw_response = response.choices[0].message.content
print("Raw response:")
print(raw_response)
print("\n" + "="*50 + "\n")

# Парсим JSON (может упасть!)
try:
    data = json.loads(raw_response)
    print("✅ Успешно распарсили JSON")
    print(json.dumps(data, ensure_ascii=False, indent=2))
except json.JSONDecodeError as e:
    print(f"❌ Ошибка парсинга: {e}")
    print("Модель вернула не валидный JSON!")


Raw response:
{
  "transactions": [{"amount": 1500.0, "category": "продукты"}],
  "answer": "Транзакция на покупку продуктов успешно извлечена."
}


✅ Успешно распарсили JSON
{
  "transactions": [
    {
      "amount": 1500.0,
      "category": "продукты"
    }
  ],
  "answer": "Транзакция на покупку продуктов успешно извлечена."
}


#### ❌ Проблемы старого подхода:

1. **Нет гарантий формата** - модель может вернуть что угодно
2. **Модель может добавить текст** до/после JSON (например: "Вот результат: {...}")
3. **Ошибки в синтаксисе** - лишние запятые, кавычки
4. **Неправильные типы** - строка вместо числа, null вместо массива
5. **Нужны костыли** - regex для извлечения JSON, try/except

#### ✅ Плюсы:
- Работает с любыми моделями (даже старыми)
- Не требует специальных параметров API

#### Вывод: **Не используйте этот подход в 2024+ году!**


---
### 1.2. Современный подход: Pydantic модели

Определяем структуру данных с валидацией через Pydantic


In [4]:
from pydantic import BaseModel, Field
from datetime import date, time
from enum import Enum
from typing import Optional

# Enum для ограничения значений
class TransactionType(str, Enum):
    INCOME = "income"      # доход
    EXPENSE = "expense"    # расход

class TransactionFrequency(str, Enum):
    DAILY = "daily"           # повседневные
    PERIODIC = "periodic"     # периодические
    ONE_TIME = "one_time"     # разовые

# Модель транзакции
class Transaction(BaseModel):
    date: date                           # дата (обязательно)
    time: Optional[time] = None          # время (опционально)
    type: TransactionType                # доход/расход (только из Enum!)
    amount: float = Field(gt=0)          # сумма > 0 (валидация!)
    frequency: TransactionFrequency      # тип транзакции
    category: str                        # категория (продукты, такси, etc)
    description: str = ""                # описание

# Модель ответа LLM
class TransactionResponse(BaseModel):
    transactions: list[Transaction]      # список транзакций (может быть [])
    answer: str                          # текстовый ответ пользователю

# Тест модели
test_data = {
    "transactions": [{
        "date": "2024-10-30",
        "time": None,
        "type": "expense",
        "amount": 1500.0,
        "frequency": "daily",
        "category": "продукты",
        "description": "Покупка продуктов"
    }],
    "answer": "Записал расход"
}

# Парсинг с валидацией
response = TransactionResponse.model_validate(test_data)
print("✅ Валидация прошла успешно!")
print(f"Транзакций: {len(response.transactions)}")
print(f"Сумма: {response.transactions[0].amount}₽")


✅ Валидация прошла успешно!
Транзакций: 1
Сумма: 1500.0₽


---
### 1.3. JSON Schema + Structured Output

**Ключевая фишка:** передаем схему модели в API → модель **гарантированно** вернет валидный JSON


In [5]:
# Генерируем JSON Schema из Pydantic модели
schema = TransactionResponse.model_json_schema()

print("JSON Schema (фрагмент):")
import json
print(json.dumps(schema, indent=2, ensure_ascii=False)[:500] + "...")


JSON Schema (фрагмент):
{
  "$defs": {
    "Transaction": {
      "properties": {
        "date": {
          "format": "date",
          "title": "Date",
          "type": "string"
        },
        "time": {
          "default": null,
          "title": "Time",
          "type": "null"
        },
        "type": {
          "$ref": "#/$defs/TransactionType"
        },
        "amount": {
          "exclusiveMinimum": 0,
          "title": "Amount",
          "type": "number"
        },
        "frequency": {
       ...


In [8]:
# Вызов LLM с structured output
system_prompt = """Ты финансовый советник. Извлекай транзакции из сообщений.

Правила:
- Извлекай только из последнего сообщения
- Если суммы нет - нет транзакции
- Категории: продукты, рестораны, такси, транспорт, развлечения, здоровье, одежда, другие
"""

user_message = "Поехал на такси за 500р, купил кофе на 200"

response = client.chat.completions.create(
    model="openai/gpt-oss-20b:free",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ],
    # 🔥 ГЛАВНОЕ: response_format с JSON Schema
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "transaction_response",
            "schema": TransactionResponse.model_json_schema(),
            "strict": True  # строгое соответствие схеме
        }
    }
)

# Парсим ответ
raw_content = response.choices[0].message.content
print("Raw JSON:")
print(raw_content[:300] + "...\n")

# Валидация через Pydantic
parsed = TransactionResponse.model_validate_json(raw_content)

print(f"✅ Получено транзакций: {len(parsed.transactions)}")
for t in parsed.transactions:
    print(f"  • {t.category}: {t.amount}₽ ({t.type})")
print(f"\nОтвет: {parsed.answer}")


Raw JSON:
{"transactions":[{"date":"2025-10-30","time":null,"type":"expense","amount":500,"frequency":"one_time","category":"такси"},{"date":"2025-10-30","type":"expense","amount":200,"frequency":"one_time","category":"продукты"}],"answer":"Transactions extracted"}...

✅ Получено транзакций: 2
  • такси: 500.0₽ (TransactionType.EXPENSE)
  • продукты: 200.0₽ (TransactionType.EXPENSE)

Ответ: Transactions extracted


#### ✅ Плюсы structured output:

1. **Гарантия формата** - всегда валидный JSON
2. **Соответствие схеме** - все поля на месте
3. **Правильные типы** - число, строка, дата
4. **Нет галлюцинаций в структуре** - только в содержимом
5. **Не нужны костыли** - парсится с первого раза

#### Параметр `strict: True`:
- Модель **не может** добавить лишние поля
- Модель **не может** пропустить обязательные поля
- Типы **строго** соответствуют схеме


---
### 1.4. Совместимость моделей и провайдеров

⚠️ **Важно:** не все модели/провайдеры одинаково поддерживают structured output!

#### Поддержка `response_format` с `json_schema`:

| Провайдер | Модели | `strict: True` | Примечания |
|-----------|--------|----------------|------------|
| **OpenAI** | gpt-4o, gpt-4o-mini | ✅ | Полная поддержка |
| **OpenRouter** | Зависит от модели | ⚠️ | Проверяйте документацию модели |




#### Fallback стратегии:
# 

1. **`strict: False`** - базовый JSON mode (модель старается, но не гарантирует)
2. **`type: "json_object"`** - вместо `json_schema` (еще проще, но без схемы!)
3. **Промпт инжиниринг** - просим JSON в промпте (старый подход)

#### 🔍 Разница между `json_object` и `json_schema`:

**`type: "json_object"`:**
- ✅ Гарантирует **валидный JSON** (синтаксически корректный)
- ❌ **НЕ проверяет структуру** - может быть любой JSON
- ❌ **НЕ проверяет типы** - строка вместо числа? Пожалуйста!
- ❌ **НЕ гарантирует поля** - может пропустить или добавить лишние
- 📝 Модель сама решает какую структуру вернуть

**`type: "json_schema"` с `strict: True`:**
- ✅ Гарантирует **валидный JSON**
- ✅ **Проверяет структуру** - точно по схеме
- ✅ **Проверяет типы** - число будет числом, строка строкой
- ✅ **Гарантирует поля** - все обязательные на месте, лишних нет
- 📝 Модель **обязана** следовать схеме

**Пример:**
```python
# json_object может вернуть:
{"result": "42", "extra_field": true}  # Строка вместо числа, лишнее поле

# json_schema вернет строго:
{"result": 42}  # Только то, что в схеме, правильного типа
```


In [9]:
# Для моделей без поддержки strict или json_schema
def call_llm_with_fallback(model: str, messages: list):
    try:
        # Попытка 1: strict JSON Schema
        return client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "response",
                    "schema": TransactionResponse.model_json_schema(),
                    "strict": True
                }
            }
        )
    except Exception as e:
        print(f"⚠️ Strict mode не поддерживается: {e}")
        try:
            # Попытка 2: базовый JSON mode
            return client.chat.completions.create(
                model=model,
                messages=messages,
                response_format={"type": "json_object"}
            )
        except Exception as e2:
            print(f"⚠️ JSON mode не поддерживается: {e2}")
            # Попытка 3: обычный вызов (надеемся на промпт)
            return client.chat.completions.create(
                model=model,
                messages=messages
            )

print("✅ Функция fallback готова для работы с разными моделями")


✅ Функция fallback готова для работы с разными моделями


#### 🔍 Как проверить поддержку?

1. **Документация провайдера** - читайте внимательно
2. **Тестирование** - пробуйте и смотрите на ошибки
3. **OpenRouter** - в описании модели указана поддержка features


---
### 1.5. In-Context Learning и Few-Shot

**Задача:** научить модель правильно извлекать транзакции через примеры

#### Теория Few-Shot Learning:

**Few-shot** = даем модели несколько примеров "вход → выход" в промпте

- **Zero-shot:** только инструкция, без примеров
- **One-shot:** 1 пример
- **Few-shot:** 2-5 примеров (обычно достаточно)
- **Many-shot:** 10+ примеров (редко нужно)

#### Правила формирования примеров:

1. **Разнообразие** - покрывайте разные edge cases:
   - Простой случай: "купил хлеб 50р"
   - Множество транзакций: "такси 500р и кофе 200р"
   - Нет транзакций: "привет, как дела?"
   - Неполная информация: "купил что-то" (без суммы)

2. **Качество** - примеры должны быть **идеальными**:
   - Правильные категории
   - Правильные типы (income/expense)
   - Правильная частота (daily/periodic/one_time)

3. **Количество** - 2-5 примеров обычно достаточно:
   - Больше ≠ лучше (увеличивается latency и cost)
   - Важнее качество, чем количество


#### Где размещать Few-Shot?

**Вариант 1: В system prompt** (рекомендуется)


In [11]:
# Few-shot примеры в system prompt (из нашего проекта)
system_prompt_with_examples = """Ты финансовый советник. Извлекай транзакции из сообщений.

Правила:
- type: "income" (доход) или "expense" (расход)
- frequency: "daily" (повседневное), "periodic" (регулярное), "one_time" (разовое)
- category: продукты, рестораны, такси, транспорт, развлечения, здоровье, одежда, другие
- amount: только положительные числа

Примеры:

Пример 1 - простая покупка:
Вход: "Купил продукты на 1500 рублей"
Выход:
{
  "transactions": [{
    "date": "2024-10-30",
    "time": null,
    "type": "expense",
    "amount": 1500.0,
    "frequency": "daily",
    "category": "продукты",
    "description": "Покупка продуктов"
  }],
  "answer": "Записал расход на продукты 1500₽"
}

Пример 2 - несколько транзакций:
Вход: "Поехал на такси за 500р, потом купил кофе на 200"
Выход:
{
  "transactions": [
    {"date": "2024-10-30", "type": "expense", "amount": 500.0, "frequency": "periodic", "category": "такси", "description": "Поездка на такси"},
    {"date": "2024-10-30", "type": "expense", "amount": 200.0, "frequency": "daily", "category": "рестораны", "description": "Кофе"}
  ],
  "answer": "Записал 2 расхода: такси 500₽ и кофе 200₽"
}

Пример 3 - нет транзакций:
Вход: "Привет, как дела?"
Выход:
{
  "transactions": [],
  "answer": "Привет! Все отлично. Чем могу помочь?"
}
"""

print("✅ Few-shot промпт готов")


✅ Few-shot промпт готов


**Вариант 2: В message history** (для динамических примеров)


In [12]:
# Few-shot как диалог
messages_with_examples = [
    {"role": "system", "content": "Ты финансовый советник. Извлекай транзакции."},
    # Пример 1
    {"role": "user", "content": "Купил продукты на 1500р"},
    {"role": "assistant", "content": '{"transactions": [{"amount": 1500.0, "category": "продукты", "type": "expense", "frequency": "daily", "date": "2024-10-30", "description": ""}], "answer": "Записал"}'},
    # Пример 2
    {"role": "user", "content": "Привет"},
    {"role": "assistant", "content": '{"transactions": [], "answer": "Привет!"}'},
    # Реальный запрос
    {"role": "user", "content": "Съел бургер за 350р"}
]

print("✅ Few-shot как история готов")


✅ Few-shot как история готов


#### Что в промпте, а что в схеме?

**В JSON Schema:**
- Структура полей
- Типы данных
- Обязательность полей
- Базовые constraint (min, max)

**В промпте:**
- Бизнес-логика (что считается доходом/расходом)
- Семантика (какие категории использовать)
- Edge cases (что делать если нет суммы)
- Few-shot примеры


In [14]:
# Практический тест с использованием openai/gpt-4o-mini через OpenRouter
test_message = "Получил зарплату 50000, купил продукты на 3000 и поехал на такси за 800"

response = client.chat.completions.create(
    model="openai/gpt-oss-20b:free",
    messages=[
        {"role": "system", "content": system_prompt_with_examples},
        {"role": "user", "content": test_message}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "transaction_response",
            "schema": TransactionResponse.model_json_schema(),
            "strict": True
        }
    },
    temperature=0.1
)

result = TransactionResponse.model_validate_json(response.choices[0].message.content)

print(f"📝 Вход: {test_message}\n")
print(f"📊 Транзакций: {len(result.transactions)}\n")
for t in result.transactions:
    print(f"  • [{t.type}] {t.category}: {t.amount}₽ ({t.frequency})")
print(f"\n💬 Ответ: {result.answer}")


📝 Вход: Получил зарплату 50000, купил продукты на 3000 и поехал на такси за 800

📊 Транзакций: 3

  • [TransactionType.INCOME] другое: 50000.0₽ (TransactionFrequency.PERIODIC)
  • [TransactionType.EXPENSE] продукты: 3000.0₽ (TransactionFrequency.DAILY)
  • [TransactionType.EXPENSE] такси: 800.0₽ (TransactionFrequency.PERIODIC)

💬 Ответ: Записал 3 транзакции: зарплата 50000₽, продукты 3000₽ и такси 800₽


---
## Часть 2: Vision Models - Анализ изображений

### Задача: распознавание чеков

Извлекаем транзакции из фото чека (OCR + structured output)

**Что делаем:**
1. Загружаем изображение чека и конвертируем в base64
2. Отправляем в vision-модель (qwen/qwen3-vl-8b-instruct)
3. Модель распознает текст (OCR) и извлекает структурированные данные
4. Получаем транзакции в формате TransactionResponse

### Vision модели умеют:
- Читать текст с изображений (чеки, выписки, скриншоты)
- Понимать контекст (что это чек из магазина, а не просто текст)
- Извлекать структурированные данные (товары, суммы, категории)


In [16]:
import base64
from pathlib import Path

# Функция для конвертации изображения в base64
def image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

# Загружаем пример чека из проекта (используйте свой путь к изображению)
receipt_path = "receipt_example.png"  # замените на путь к вашему чеку

# Конвертируем в base64
image_b64 = image_to_base64(receipt_path)
print(f"✅ Изображение загружено: {len(image_b64)} символов base64")
print(f"📦 Размер: ~{len(image_b64) * 3 // 4 / 1024:.1f} KB")

# Для демонстрации создадим заглушку
print("📝 Для работы с изображениями нужен файл чека")


✅ Изображение загружено: 1187736 символов base64
📦 Размер: ~869.9 KB
📝 Для работы с изображениями нужен файл чека


In [22]:
# System prompt для vision модели (из нашего проекта)
vision_system_prompt = """Ты персональный финансовый советник. Помогаешь пользователю вести учет доходов и расходов.

ВАЖНО: Транзакции извлекай ТОЛЬКО из присланного изображения (чека, скриншота). Не используй информацию из предыдущих сообщений для создания транзакций.

Твоя задача:
1. Распознать текст на изображении (OCR)
2. Извлечь финансовые транзакции из изображения (чеки, выписки, скриншоты платежей)
3. Для каждой транзакции создать объект со следующими полями (используй ТОЧНО эти названия полей на английском):
   - "date" (обязательно, формат YYYY-MM-DD строкой)
   - "time" (если указано на изображении, формат HH:MM:SS строкой, иначе null)
   - "type" (обязательно, строка: "income" для дохода или "expense" для расхода)
   - "amount" (обязательно, положительное число float)
   - "frequency" (обязательно, строка: "daily" для повседневных, "periodic" для периодических, "one_time" для разовых)
   - "category" (обязательно, строка на русском: продукты, рестораны, такси, транспорт, образование, путешествия, развлечения, здоровье, одежда, другие или новая категория по смыслу)
   - "description" (строка, подробная информация о товарах, услугах, источнике дохода, контрагенте и т.п., может быть пустой "")
4. Дать краткий комментарий о найденных транзакциях в поле "answer"

Категории: продукты, рестораны, такси, транспорт, образование, путешествия, развлечения, здоровье, одежда, другие. Можешь предлагать новые категории, если они подходят лучше.

Твой ответ должен быть в формате JSON с двумя полями:
- "transactions" - массив транзакций (ОБЯЗАТЕЛЬНО, всегда должен быть в ответе, даже если пустой [])
- "answer" - текстовый ответ пользователю (обязательно)

ВАЖНО: Если на изображении нет информации о транзакциях или не удалось извлечь транзакции, ВСЕ РАВНО верни поле "transactions" со значением [] (пустой массив). Поле transactions всегда должно присутствовать в ответе!


"""

# Вызов vision модели (если есть изображение)
# Пример с qwen/qwen-2-vl-7b-instruct

try:
    response = client.chat.completions.create(
        model="qwen/qwen3-vl-8b-instruct",  # vision модель на OpenRouter
        messages=[
            {"role": "system", "content": vision_system_prompt},
            {
                "role": "user",
                "content": [
                    # 🔥 Формат для изображений
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_b64}"
                        }
                    },
                    {
                        "type": "text",
                        "text": "Извлеки транзакции из этого чека"
                    }
                ]
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "transaction_response",
                "schema": TransactionResponse.model_json_schema(),
                "strict": True
            }
        }
    )
    # Парсим ответ вручную (модель вернет JSON в тексте)
    content = response.choices[0].message.content
    print("📝 Ответ модели:")
    print(content)
    
    # Пытаемся распарсить JSON из ответа
    import json
    result = TransactionResponse.model_validate_json(content)
    print(f"\n📊 Распознано транзакций: {len(result.transactions)}")
    
    for i, tx in enumerate(result.transactions, 1):
        print(f"{i}. {tx.description}: {tx.amount} руб. ({tx.category})")
        
except Exception as e:
    print(f"⚠️ Ошибка при обработке изображения: {e}")
    print("\n💡 Возможные причины:")
    print("- Vision модель не поддерживает structured output (response_format)")
    print("- Нужно парсить JSON из текстового ответа")
    print("- Попробуйте другую модель (gpt-4o, claude-3-sonnet)")


📝 Ответ модели:
{
  "transactions": [
    {
      "date": "2015-11-12",
      "time": null,
      "type": "expense",
      "amount": 1720.5,
      "frequency": "one_time",
      "category": "здоровье",
      "description": "Покупка лекарств: Аква Марис, Карнолис, Пластырь СанитаПласт, Хлоргексидин. Оплачено картой."
    }
  ],
  "answer": "Из чека извлечена одна транзакция — расход на лекарства в сумме 1720.50 руб. Категория: здоровье."
}

📊 Распознано транзакций: 1
1. Покупка лекарств: Аква Марис, Карнолис, Пластырь СанитаПласт, Хлоргексидин. Оплачено картой.: 1720.5 руб. (здоровье)


### Особенности Vision моделей:

#### Формат сообщения:
```python
{
  "role": "user",
  "content": [
    {"type": "image_url", "image_url": {"url": "data:image/jpeg;base64,..."}},
    {"type": "text", "text": "вопрос про изображение"}
  ]
}
```

#### Популярные vision модели:
- **GPT-4o / GPT-4o-mini** (OpenAI) - лучшее качество OCR
- **Claude 3 Sonnet/Opus** (Anthropic) - отличный OCR
- **Qwen2-VL** (Alibaba) - бесплатная на OpenRouter
- **LLaVA** - локальная open-source


---
## Часть 3: Локальные модели - Теория

### Зачем запускать модели локально?

✅ **Плюсы:**
- Бесплатно (после покупки железа)
- Приватность - данные не покидают устройство
- Нет ограничений API (rate limits)
- Работает offline

❌ **Минусы:**
- Нужно мощное железо (GPU с 8+ GB VRAM)
- Качество моделей хуже коммерческих
- Нужно управлять моделями самому
- Медленнее облачных API


### Сравнение способов локального запуска

| Способ | Простота | Производительность | API | GPU | Use Case |
|--------|----------|-------------------|-----|-----|----------|
| **Transformers** | ⭐⭐ | ⭐⭐⭐ | ❌ Нет | Да | Эксперименты, кастомизация |
| **llama.cpp** | ⭐⭐⭐ | ⭐⭐⭐⭐ | ❌ Нет | Нет (CPU) | Легкие модели на CPU |
| **Ollama** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ✅ OpenAI | Опционально | Разработка, личное use |
| **LM Studio** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ✅ OpenAI | Опционально | Новички, GUI |
| **vLLM** | ⭐⭐ | ⭐⭐⭐⭐⭐ | ✅ OpenAI | Да | Production, high load |

#### Детали:

**1. Transformers (HuggingFace)**
- Нативная Python библиотека
- Полный контроль над моделью
- Сложнее использовать
- Нет готового OpenAI API

**2. llama.cpp**
- C++ библиотека, оптимизирована для CPU
- Работает на MacBook (Apple Silicon оптимизация)
- Квантизация моделей (4-bit, 5-bit)
- CLI инструмент

**3. Ollama** 🌟
- Самый простой способ для разработки
- OpenAI-совместимый API из коробки
- Автоматическое управление моделями
- Отлично для Mac/Linux

**4. LM Studio**
- GUI приложение (Windows/Mac/Linux)
- Drag & drop загрузка моделей
- Встроенный OpenAI-совместимый сервер
- Идеально для новичков

**5. vLLM**
- Production-grade решение
- Максимальная производительность (PagedAttention)
- OpenAI API
- Требует GPU и опыта


### Какой выбрать?

**Для обучения/экспериментов:**
- → Ollama или LM Studio

**Для разработки:**
- → Ollama (простота + OpenAI API)

**Для production:**
- → vLLM (производительность)

**Для CPU (без GPU):**
- → llama.cpp
- → Ollama (тоже работает на CPU)

**Для кастомизации:**
- → Transformers (полный контроль)
  - Нужно изменить архитектуру модели
  - Требуется fine-tuning на своих данных
  - Необходим доступ к внутренним слоям модели
  - Хотите экспериментировать с промптами на уровне токенов
  - Нужна интеграция с PyTorch/TensorFlow пайплайнами


---
## Часть 4: Локальные модели - Практика

### 4.1. Transformers (HuggingFace) - нативный подход


In [23]:
# Установка (закомментировано - может занять время)
# %pip install -q transformers torch accelerate

print("📦 Для использования Transformers нужны: transformers, torch, accelerate")


📦 Для использования Transformers нужны: transformers, torch, accelerate


In [25]:
# Пример загрузки и использования модели (закомментировано - требует GPU/много времени)
"""
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Загружаем легкую модель (пример: TinyLlama 1.1B)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"Загружаем модель {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # экономим память
    device_map="auto"  # автоматически на GPU/CPU
)

print(f"✅ Модель загружена на {model.device}")

# Генерация текста
prompt = "Extract transactions from: I spent 50 dollars on groceries."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.7)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)
"""

print("✅ Transformers закомментирован")


✅ Transformers закомментирован


#### ✅ Плюсы Transformers:
- Полный контроль (temperature, sampling, etc)
- Доступ к внутренним состояниям модели
- Можно кастомизировать (fine-tuning, PEFT)
- Огромный выбор моделей на HuggingFace Hub

#### ❌ Минусы:
- Нет OpenAI API из коробки
- Structured output сложнее (нужны guided generation библиотеки)
- Больше кода для простых задач
- Нужно понимание ML (tokenizer, device management)

#### 💡 Когда использовать:
- Эксперименты с моделями
- Fine-tuning
- Нужен полный контроль


---
### 4.2. Ollama - простота + OpenAI API

#### Установка:
```bash
# Mac/Linux
curl -fsSL https://ollama.com/install.sh | sh

# Windows
# Скачайте с https://ollama.com/download
```

#### Загрузка модели:
```bash
# В терминале
ollama pull llama3.2:3b  # легкая модель 3B параметров
# или
ollama pull qwen2.5:7b   # более мощная 7B
```


In [ ]:
# Ollama автоматически запускает сервер на localhost:11434
# Используем OpenAI client!

from openai import OpenAI

# Подключаемся к Ollama через OpenAI API
ollama_client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama OpenAI-совместимый endpoint
    api_key="ollama"  # любое значение (Ollama не требует ключ)
)

# Простой вызов (требует запущенный Ollama с моделью)
"""
response = ollama_client.chat.completions.create(
    model="llama3.2:1b",  # имя модели из `ollama list`
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Extract transactions from: bought coffee for $5"}
    ],
    temperature=0.1
)

print(response.choices[0].message.content)
"""

print("✅ Ollama client готов (требует запущенный Ollama сервер)")


There is only one transaction mentioned:

1. Bought coffee for $5
✅ Ollama client готов (требует запущенный Ollama сервер)


#### ✅ Плюсы Ollama:
- **Очень простая установка** - одна команда
- **OpenAI-совместимый API** - код не меняется
- **Автоматическое управление** моделями (`ollama pull`, `ollama list`)
- **Оптимизация** - использует llama.cpp под капотом
- **Работает на CPU** (медленно) и GPU

#### ❌ Минусы:
- Меньше контроля над параметрами модели
- Ограниченный выбор моделей (только из Ollama Registry)

#### 💡 Когда использовать:
- **Разработка** - идеально для локального тестирования
- **Личные проекты** - простота важнее производительности
- **Прототипирование** - быстро проверить идею


---
### 4.3. LM Studio - GUI для моделей

#### Установка:
1. Скачайте с https://lmstudio.ai
2. Установите приложение
3. Запустите LM Studio

#### Использование:
1. **Загрузка модели:**
   - Вкладка "Discover" → поиск модели
   - Выберите квантизацию (Q4, Q5, Q8)
   - Нажмите "Download"

2. **Запуск сервера:**
   - Вкладка "Local Server"
   - Выберите модель
   - "Start Server" (по умолчанию: localhost:1234)


In [30]:
# Подключаемся к LM Studio (тот же OpenAI client!)
lmstudio_client = OpenAI(
    base_url="http://localhost:1234/v1",  # LM Studio по умолчанию
    api_key="lm-studio"  # любое значение
)

# Вызов (требует запущенный LM Studio с загруженной моделью)
#"""
response = lmstudio_client.chat.completions.create(
    model="qwen2.5-3b-instruct",  # LM Studio автоматически использует загруженную модель
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Extract transactions from: bought coffee for $5"}
    ],
    temperature=0.7,
    max_tokens=200
)

print(response.choices[0].message.content)
#"""

print("✅ LM Studio client готов (требует запущенный LM Studio сервер)")


The extracted transaction is:
- Bought coffee for $5
✅ LM Studio client готов (требует запущенный LM Studio сервер)


#### ✅ Плюсы LM Studio:
- **GUI** - не нужна командная строка
- **Drag & drop** - загрузка моделей в 2 клика
- **Встроенный чат** - протестировать модель без кода
- **OpenAI API** из коробки
- **Квантизация** - выбирайте баланс качество/скорость
- **Cross-platform** - Windows, Mac, Linux

#### ❌ Минусы:
- **Не для production** - GUI приложение
- Меньше гибкости в настройках
- Один сервер = одна модель за раз
- `strict` JSON Schema не поддерживается

#### 💡 Когда использовать:
- **Новички** - самый простой старт
- **Эксперименты** - быстро попробовать модель
- **Демо** - показать клиенту возможности


---
### 4.4. vLLM - production-grade решение

**vLLM** - самый быстрый inference engine для LLM

#### Установка:
```bash
# Требуется CUDA GPU!
pip install vllm
```

#### Запуск сервера:
```bash
# В терминале
python -m vllm.entrypoints.openai.api_server \
    --model meta-llama/Llama-2-7b-chat-hf \
    --port 8000 \
    --tensor-parallel-size 1  # количество GPU
```


In [31]:
# Подключаемся к vLLM (требует запущенный vLLM сервер)
vllm_client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123"  # можно настроить auth
)

"""
response = vllm_client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",  # имя модели из HuggingFace
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is 2+2?"}
    ],
    temperature=0.7,
    max_tokens=100
)

print(response.choices[0].message.content)
"""

print("✅ vLLM client готов (требует запущенный vLLM сервер с GPU)")


✅ vLLM client готов (требует запущенный vLLM сервер с GPU)


#### ✅ Плюсы vLLM:
- **Максимальная производительность** - PagedAttention, continuous batching
- **Production-ready** - многопоточность, async, метрики
- **OpenAI API** совместимость
- **Guided generation** - гарантированный structured output (JSON schema, regex, grammar)
- **Tensor parallelism** - распределение на несколько GPU
- **High throughput** - обработка множества запросов

#### ❌ Минусы:
- **Требует GPU** (NVIDIA CUDA)
- Сложнее настройка
- Больше памяти требуется
- Документация может быть сложной для новичков

#### 💡 Когда использовать:
- **Production deployment**
- **High load** - много пользователей
- **Latency critical** - нужна скорость
- **Batch processing** - обработка больших объемов


---
## Заключение

### Сравнение подходов

| Подход | Простота | Качество | Скорость | Cost | Use Case |
|--------|----------|----------|----------|------|----------|
| **OpenAI API** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | 💰💰💰 | Production, высокое качество |
| **OpenRouter** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | 💰💰 | Разнообразие моделей |
| **Ollama** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ | Free | Разработка, личные проекты |
| **LM Studio** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ | Free | Эксперименты, новички |
| **vLLM** | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 💰 (GPU) | Production, high load |
| **Transformers** | ⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ | 💰 (GPU) | Кастомизация, ML |


### Рекомендации

#### Для обучения:
1. Начните с **OpenRouter** (простой API, много моделей)
2. Попробуйте **Ollama** (локально, бесплатно)
3. Изучите **Transformers** (понимание как работает под капотом)

#### Для разработки:
1. **Ollama** для локального тестирования
2. **OpenRouter/OpenAI** для качества в dev окружении
3. Используйте **structured output** везде где возможно

#### Для production:
1. **OpenAI API** если budget позволяет (качество + надежность)
2. **vLLM** если нужна скорость и есть GPU инфраструктура
3. **Ollama** для internal tools (простота + бесплатно)

### Ключевые принципы:

1. **Всегда используйте structured output** (не JSON в промпте!)
2. **Few-shot примеры** улучшают качество на 20-30%
3. **Schema hints** в промпте помогают модели понять ожидания
4. **Проверяйте совместимость** модели с `strict` mode
5. **OpenAI client** работает почти везде - единый код!


---

Удачи! 🚀

Автор: Смирнов Сергей https://t.me/smirnoff_ai
